# <font color='black'><center><center></font> 
![title](img/All.png) 
# <font color='white'><center><center></font> 
# <font><center>Artificial Intelligence<center></font>
## <font ><center>CA4: MACHINE LEARNING<center></font>
## <center>Teacher: Dr. Fadaei<center>
### <center>Student Name: Mohammadali Shakerdargah<center>
### <center>Student Number: 810098002<center>

### <font>Purpose:</font>
    In this project we are going to use "scikit-library" to perform machine learning algorithms on given data so that we can fit a model to estimate each vehicle's price.
    
### <font>What  we did in this Project:</font>
    In this project we are going to had a training dataset and a test data set to evaluate our model we used "Naive Bayes" to build a classifier model to predict the category of a book using a description and name of the book
    

In [ ]:
!nvidia-smi -L

In [ ]:
!pip install hazm
!pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade
!pip install arabic-reshaper
!pip install python-bidi

In [ ]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
from numpy import *
import math
import operator
import matplotlib.pyplot as plt
import arabic_reshaper
from bidi.algorithm import get_display
import sklearn
from sklearn import feature_selection
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# data = pd.read_csv("/content/drive/MyDrive/Colab_Documents/AI/CA4/vehicles.csv")
data = pd.read_csv("vehicles.csv")
data

In [ ]:
data.info()

# Phase 1: Preprocessing and Visualization

- First I got rid of those NAN rows. 
- Then, I changed brands from categorical to numerical values (We only had 26 specific brands). 
- After erasing NAN rows and changing brands from categorical to numerical, I decided to cleanse my data. In order to cleanse data I used "hazm" library.
    - it is formal that it is better to work with root of each word. It tells us that prefixes and postfixes are not that necessary for us to participate them in our process.And sometimes they may even reduce our accuracy because all versions of a word are telling us the same thing and almost same info,So we will use 2 methods called "Stemming" and "lemmatization". these two methods share the same goal, and the goal is to remove unnecessary prefixes and postfixes from a word. I prefered to use "lemmatization" because found it beter to use so we won't have (آسمان -> آسم) . It works
        - lemmatizer.lemmatize('می‌روم') -> 'رفت#رو'
    - I could have gone with "Stemming" too and it works like this: 
        - like this: stemmer.stem('کتاب‌ها') -> 'کتاب'
- Then, I tried to change category of the cars into numerical which containes only light cars -> 0
- I scaled "Mileage" with MinMaxScaler so that we wouldn't have our data in a  big range.


In [ ]:
data = data[pd.notnull(data['brand'])]
data = data.reset_index(drop=True)
data.info()

In [ ]:
def brandCategorizer(data, brand, categories):
    for i in range(len(data[brand])):
        brand_i = data[brand][i]
        if(brand_i in categories):
            data[brand][i] = categories.get(brand_i)
    return data

In [ ]:
uniqueCategories = data['brand'].unique()
categories ={}
for i in range(len(uniqueCategories)):
    categories[uniqueCategories[i]] = i
for category in categories:
    print("Mapped: ",category," -> ", categories.get(category))
data = brandCategorizer(data, 'brand', categories)
data

In [ ]:
def dataCleanse(df,col):
    normalizer = Normalizer()
    lemmatizer = Lemmatizer() 
    for i in range(len(df[col])):
        df[col][i] = normalizer.normalize(df[col][i])
            
    punctuations = set('''!()-[]{};:'"\,٬٫،<>./?@#$%٪^&*×ـ+=_~.؟؛«»|…''') #Set of unnecessary punctuations
    for i in range(len(df[col])):
        without_punctuations = ""
        for char in df[col][i]:
            if(char not in punctuations):
                without_punctuations = without_punctuations + char
        df[col][i] = word_tokenize(without_punctuations)
        
    stops = set(stopwords_list()) #List of unnecessary words
    for i in range(len(df[col])):
        wordsWithoutStops = []
        for j in range(len(df[col][i])):
            if(df[col][i][j] not in stops):
                wordsWithoutStops.append(lemmatizer.lemmatize(df[col][i][j]))
        df[col][i] = wordsWithoutStops
    return df

In [ ]:
data_cleansed = dataCleanse(data, 'description')
data_cleansed = dataCleanse(data_cleansed, 'title')

In [ ]:
data_cleansed

In [ ]:
def categorizer(data, brand, categories):
    for i in range(len(data[brand])):
        brand_i = data[brand][i]
        if(brand_i in categories):
            data[brand][i] = categories.get(brand_i)
    return data

In [ ]:
unique_Categories_cars = data_cleansed['category'].unique()
categories_cars ={}
for i in range(len(unique_Categories_cars)):
    categories_cars[unique_Categories_cars[i]] = i
for category in categories_cars:
    print("Mapped: ",category," -> ", categories_cars.get(category))
data_cleansed = categorizer(data_cleansed, 'category', categories_cars)
data_cleansed

In [ ]:
MMScaler = MinMaxScaler()
data_cleansed[['mileage']] = MMScaler.fit_transform(data_cleansed[['mileage']])
data_cleansed

In [ ]:
data_cleansed_withOut_price = data_cleansed.drop(columns = ['price'])
data_cleansed_withOut_price